# Spectra acquisiton
Control both the stepping motor and the camera from this notebook.<br>
Make a loop to measure a wide-range spectrum automatically.

NOTE: only one connection to COM port is possible at a time. Shut down other connections, either in python or dedicated software.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import time, os, logging
from FLI import FLI
import numpy as np
import xarray as xr
from os.path import join

In [3]:
%run thr640.py
%run imaging_tool.py

In [4]:
logger = logger
logger.setLevel(50) # 10 - DEBUG 50 - CRITICAL 20 - INFO
camera = FLI()
step_motor = THR640() # Can only have one instance.

In [5]:
basepath = r'\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210106_test'
os.path.exists(basepath)

True

In [6]:
join(basepath,'a.nc')

'\\\\10.249.254.51\\Public\\Kuzmin\\Experiments-Leprecon\\Spectrometer\\20210106_test\\a.nc'

In [9]:
exposure = 5.0 * 1e3 # ms
vbin = 1
attributes = {
    "temperature": camera.getTemperature(),
    "device_status": camera.getDeviceStatus(),
    "exposure": exposure,
    "frame_type": "light",
    "motor_coordinate":"",
    "slit_width": "0.3mm",
    "dennryu": 1
}

#R = [100000 + i*5000 for i in range(101)]
R = [630000 + i*5000 for i in range(4)]
#R = [380000 + i*5000 for i in range(25)]
#R = [500000 + i*5000 for i in range(5)]
#R = [300000 + i*5000 for i in range(101)]
# R = [430000 + i*5000 for i in range(8)] #fulcher-alpha
# R = [430000] # 
#R = [700000 + i*10000 for i in range(10)]
#R = [740000 + i*5000 for i in range(5)]
for r in log_progress(R):
    step_motor.goto(r)
    step_motor.waitUntilReady()
    attributes['motor_coordinate'] = r
    
    camera.setExposureTime(exposure)
    camera.setVBin(vbin)
    camera.setImageArea(10,0,2058,512//vbin)
    time.sleep(.5)
    camera.exposeFrame()
    time.sleep(.1)
    # load image
    image_data = camera.grabFrame(out=np.empty((512//vbin,2048), np.uint16))
    # convert to xarray
    data = xr.DataArray(
        image_data, dims=["y", "x"], coords={"image_counter": 0}, attrs=attributes
    )
    # save data as NetCDF file
    filepath = join(basepath,f'H2_standard_{r}_miura.nc')
    print(filepath)
    data.to_netcdf(filepath)
print('Done.')

\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210106_test\H2_standard_630000_miura.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210106_test\H2_standard_635000_miura.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210106_test\H2_standard_640000_miura.nc
\\10.249.254.51\Public\Kuzmin\Experiments-Leprecon\Spectrometer\20210106_test\H2_standard_645000_miura.nc
Done.


In [8]:
[430000 + i*5000 for i in range(10)]

[430000,
 435000,
 440000,
 445000,
 450000,
 455000,
 460000,
 465000,
 470000,
 475000]